# Convolutional Neural Networks

## Applied Machine Learning

## Name:Angelica LOZANO GOMEZ

### Overview

The [`CIFAR10` dataset](https://keras.io/api/datasets/cifar10/) contains 50,000 32 $\times$ 32 colored images of the things listed below. 

| Label    | Description |
| -------- | -------|
| 0 | airplane    |
| 1 | automobile     |
| 2 | bird    |
| 3 | cat    |
| 4 | deer    |
| 5 | dog    |
| 6 | frog    |
| 7 | horse    |
| 8 | ship    |
| 9 | truck    |

The `base_model` contains a CNN that was trained on over 17,000 CIFAR10 images, none of which contained a horse, ship, or truck. In this assignment, you will use the bottleneck layers of the `base_model`, along with transfer learning to build a network that can classify all 10 categories of the images in the CIFAR10 dataset.

### Install Tensorflow 2.9.2

In [4]:
# CODE
import subprocess
import sys
from IPython.display import clear_output
def update_libraries():
    tf_desired_version = "2.9.2"

    try:
        import tensorflow as tf
        tf_installed_version = tf.__version__
    except ImportError:
        tf_installed_version = None

    # Update pip to the latest version
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'])
    print("Pip updated!")
    
    #Check for the right version
    if tf_installed_version != tf_desired_version:
        print(f"Current TensorFlow version: {tf_installed_version}. Installing version {tf_desired_version}...")
        
        # Uninstall the current TensorFlow version
        subprocess.check_call([sys.executable, '-m', 'pip', 'uninstall', '-y', 'tensorflow'])
        
        # Install the desired TensorFlow version
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', f'tensorflow=={tf_desired_version}'])

        clear_output()
        print("TensorFlow version {tf_desired_version} installed successfully")
        
    else:
        print(f"TensorFlow version {tf_desired_version} is already installed.")    


    print(f"Please restart your kernel to apply the changes.")
    
update_libraries()

Pip updated!
TensorFlow version 2.9.2 is already installed.
Please restart your kernel to apply the changes.


In [5]:
# common imports
import numpy as np
import pandas as pd

### Data and Model Prep

Load the `base_model` and then run the following cell to load the data.

In [7]:
# CODE
from tensorflow.keras.models import load_model
base_model = load_model('base_model3.h5')

Metal device set to: Apple M2


2025-04-18 22:30:05.816380: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-18 22:30:05.816786: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
# CODE -- takes almost 3 minutes to run
x_train = np.loadtxt('data/x_train.txt').reshape(32516, 32, 32, 3)
y_train = np.loadtxt('data/y_train.txt')
x_test = np.loadtxt('data/x_test.txt').reshape(6512, 32, 32, 3)
y_test = np.loadtxt('data/y_test.txt')

Let's test the `base_model`. The below code loads and displays a sample image of a ship from the test dataset. Use the `base_model` to make a prediction about the object contained in the sample image.

In [ ]:
# CODE
%matplotlib inline
import matplotlib.pyplot as plt
x = x_test[0]

print(y_test[0])
plt.xticks([])
plt.yticks([])
plt.imshow(x)

In [ ]:
# CODE
from tensorflow.keras.preprocessing import image

# preprocess the image
x = image.img_to_array(x_test[0])
x = np.expand_dims(x, axis=0)

# CODE
from tensorflow.keras.models import Model
model = Model(inputs=base_model.layers[0].input, outputs=base_model.layers[3].output)

What does the `base_model` predict that the sample image is? Let's use transfer learning to create a domain-specific model that can classify images of horses, ships, and trucks in addition to the rest of the images that the `base_model` was trained on.

The below code extracts the bottleneck layers from the `base_model`.

In [ ]:
# CODE
from tensorflow.keras.models import Model

model = Model(inputs = base_model.layers[0].input, outputs = base_model.layers[3].output)

### Transfer Learning

Create the classification layers for transfer learning and train them.

In [ ]:
# CODE
from tensorflow.keras import layers, models, Input

# Input shape is the output of the CNN model (6, 6, 64)
classification_input = Input(shape=(6, 6, 64))
x = layers.Flatten()(classification_input)
x = layers.Dense(128, activation='relu')(x)
classification_output = layers.Dense(10, activation='softmax')(x)

classification = Model(inputs=classification_input, outputs=classification_output)

In [ ]:
# CODE
classification.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

In [ ]:
# CODE -- takes about 2 minutes to run
# First get the features from the CNN model
features_train = model.predict(x_train)

# Then train the classifier on these features
history = classification.fit(features_train, y_train,
                             epochs=10,
                             batch_size=64,
                             validation_split=0.2)


Plot the training and validation accuracy.

In [ ]:
# CODE
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Check to ensure the new model correctly classifies the sample image of a ship.

In [ ]:
# CODE
from tensorflow.keras.preprocessing import image

# Reload and preprocess the sample image
x_sample = image.img_to_array(x_test[0])
x_sample = np.expand_dims(x_sample, axis=0)

# Get CNN features from the sample image using the base CNN model
features_test = model.predict(x_sample)

# Predict using the classification model
new_prediction = classification.predict(features_test)
print("Predicted class with new model:", np.argmax(new_prediction))

### Export Models for codegrade evaluation

Export Classification and CNN models

In [ ]:

#export your classification model
classification.save('classification_model.h5')

#export your CNN model
model.save('CNN_model.h5')